# Klassifikation von Hunderassen mit SVMs
Dieses Notebook nutzt zur Klassifizierung von Hunderassen eine Methode des klassischen maschinellen Lernens, die Support Vector Machine (SVM).

## 1. Laden der Daten und Aufsplitten der Daten in Trainings- und Testdaten

### Herunterladen der Daten

Bevor das Notebook ausgeführt werden kann, muss der `Images`-Ordner des [Stanford Dog Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) heruntergeladen werden und im selben Verzeichnis wie dieses Notebook im Unterordner `dataset` gesichert werden. Die resultierende Dateistruktur sollte so aussehen (Ausschnitt):
```
.
├── svm.ipynb
├── dataset
│   ├── Images
│   │   ├── n02113799-standard_poodle
│   │   ├── ...
```

Genau genommen genügen die `Images`-Unterordner

- `n02113799-standard_poodle`
- `n02113978-Mexican_hairless`
- `n02115641-dingo`
- `n02115913-dhole`
- `n02116738-African_hunting_dog`

da dies die fünf Rassen sind, die von der SVM klassifiziert werden sollen.

### Splitting

Da in jeder Klasse ausreichend Bilder zur Verfügung stehen, um Test- und Trainingsdaten bei der Kreuzvalidierung sínnvoll trennen zu können, ist dieser Vorverarbeitungsschritt prinzipiell nicht nötig. Dennoch soll an dieser Stelle gezeigt werden, wie mithilfe des eigens entwickelten Split-Helpers die Daten aufgetrennt werden.

**Hinweis:** Dieser Schritt läuft nur dann ohne Fehler, wenn die Ordner `./dataset/Train` und `./dataset/Test` noch nicht existieren. So wird sichergestellt, dass die Daten nicht mehrfach in dieselben Ordner gesplittet werden und dadurch Duplikate entstehen. Um diese Fehler zu vermeiden, kann für nachfolgende Durchläufe die Variable `SPLIT_DATA` auf `False` gesetzt werden.

In [ ]:
from helpers.split import train_test_split

SPLIT_DATA = False  # Set True to create a data split (Make sure ./dataset/Test and ./dataset/Train directories do not exist)

if SPLIT_DATA:
    train_test_split()
    print("\n\n --- Splitting done ---")

Daneben steht im Helpers-Package auch die Funktion `load_dataset` zur Verfügung, mit der direkt aus dem `Images`-Ordner oder aus den vorbereiteten Trainings- und Test-Verzeichnissen die Bilder und zugehörigen Hunderassen eingelesen werden können. Dabei durchlaufen die Bilder eine Vorverarbeitung, in der sie auf standardmäßig 256x256 Pixel skaliert werden. Kleinere Bilder sowie die kürzeren Seiten nach der das Seitenverhältnis bewahrenden Skalierung werden schwarz aufgefüllt. Heraus kommt ein Tupel, mit den Bilddaten als `ndarray` und den Labels als `ndarray`. Für die SVM muss das Format der Labels allerdings noch angepasst werden, damit nicht mehr pro Klasse eine Komponente gesetzt wird. Standardmäßig entspricht das Label-Array vom Aufbau her nämlich der folgenden Tabelle (ohne Überschriften), die SVM erwartet allerdings einen Vektor (also 1xn statt 5xn).

| Pudel | Mexican | Dingo | Dhole | African |
|:------|:--------|:------|:------|:--------|
| 1     | 0       | 0     | 0     | 0       |
| 0     | 0       | 1     | 0     | 0       |
| 0     | 1       | 0     | 0     | 0       |
| 0     | 0       | 0     | 0     | 1       |
| 0     | 0       | 0     | 1     | 0       |

Da jedes Bild nur einer Hunderasse zugeordnet ist, die Indizes der Rassen sich nicht ändern und die einzige Spalte mit dem Maximalwert 1 jeweils die dem Bild zugeordnete Hunderasse ist, kann der Index mit dem maximalen Wert als Klassenlabel für die SVM verwendet werden.

In [ ]:
from helpers.dataloader import load_dataset
import numpy as np

# load all (unsplitted) Image Data. Alternatively, change the path to use Train/Test splits
train_img, train_label_matrix = load_dataset("./dataset/Train/Images")
test_img, test_label_matrix = load_dataset("./dataset/Test/Images")
images = np.concatenate((train_img, test_img))
label_matrix = np.concatenate((train_label_matrix, test_label_matrix))

# reformat labels
labels = np.argmax(label_matrix, axis=1)
print(labels)

## 2. Merkmalsextraktion

Damit die SVM trainiert werden kann, müssen aus den Bildern relevante Features extrahiert werden. Im Folgenden werden diese kurz vorgestellt.

### Körperform

Um die Körper-/Kopfform des Hundes zu bestimmen, wird eine Hough-Transformation durchgeführt. Dafür werden die durchschnittlichen Radien und die Standardabweichung der 10 signifikantesten Kreise als Features gewählt, da sie Rückschlüsse darauf erlauben, ob der Hund eher rund oder kantig ist.
Ebenfalls wird die Anzahl der Ecken als Merkmal für den Körperbau verwendet, um beispielsweise spitze oder runde Ohren besser auseinanderhalten zu können.

### Fell

Zur Erkennung, ob ein Hund Fell hat (wie ein Pudel) oder nicht (wie ein Mexican Hairless) wird ein empfindlich eingestellter Canny-Kantendetektor eingesetzt. Außerdem wird die Entropie des Bildes bestimmt, um über die "Unruhe" Rückschlüsse auf die Beschaffenheit des Fells zu ziehen.

### Farbe

Die Durchschnittsfarbe des Bilds, ein Grauwert- und ein RGB-Histogramm sollen dabei helfen, den Hund anhand der Fellfarbe zu identifizieren.

### Ergebnis

Das Modell wurde mit verschiedenen Kombinationen der Featurevektoren getestet, das beste Ergebnis basierend auf Genauigkeit und benötigter Verarbeitungszeit konnte durch die Verwendung von Grauwerthistogramm und Entropie erzielt werden. Daher sind nur diese Funktionen im Notebook enthalten. Die anderen, weniger nützlichen Featureextraktionen finden sich in `various_features.py`.

In [ ]:
from skimage.measure import shannon_entropy
from skimage.color import rgb2gray


def extract_features_histogram(image, bins=32):
    image = rgb2gray(image)
    hist, _ = np.histogram(image, bins=bins, range=(0, 1))
    return hist / hist.sum()  # Normalise


def extract_image_entropy(image):
    image = rgb2gray(image)
    entropy_value = shannon_entropy(image)
    return [entropy_value]

Anhand dieser Funktionen kann nun der Merkmalsvektor für ein Bild bestimmt werden. Das Bild soll an dieser Stelle bereits als `ndarray` vorhanden sein.

In [ ]:
def extract_features(image):
    fv = []
    fv.extend(extract_features_histogram(image))
    fv.extend(extract_image_entropy(image))
    return fv

Mit `extract_features` wird nun für jedes Bild der eingelesenen Daten der Merkmalsvektor berechnet. So ergibt sich aus `features` und `labels` das fertig vorbereitete Datenset für die SVM.

In [ ]:
features = []
for index, image in enumerate(images):
    if index % 50 == 0:
        print("Processing img", index)
    features.append(extract_features(image))

## Merkmalsreduktion und Hyperparameteroptimierung

Bei der Merkmalsreduktion werden die Verfahren PCA und LDA verglichen. Da das Training der SVM schnell geht, wird für beide Möglichkeiten eine Hyperparameteroptimierung vorgenommen und am Ende die jeweils beste Accuracy verglichen.

Dafür werden zunächst Pipelines für die beiden Möglichkeiten angelegt, jeweils mit dem StandardScaler für die Merkmalsskalierung und einer SVM. Bei der SVM wird der Random State jeweils auf denselben Wert gesetzt, um vergleichbare Ergebnisse zu erhalten. Dadurch ist der einzige Unterschied die PCA bzw. LDA. Außerdem werden die Hyperparameterraster für die Hyperparameteroptimierung mit GridSearch definiert. Diese unterscheiden sich, damit für jede Pipeline die jeweils besten Hyperparameter ermittelt werden können. Über die Aufgabenstellung hinaus wurde auch die Anzahl der

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# Pipelines
pipelines = {
    'PCA': Pipeline([
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=5)),
        ("svc", SVC(kernel="rbf", random_state=42))
    ]),
    'LDA': Pipeline([
        ("scaler", StandardScaler()),
        ("lda", LDA()),
        ("svc", SVC(kernel="rbf", random_state=42))
    ]),
}

# Hyperparameter-Raster
# C controls the margin tolerance of an SVM. Gamma defines the range of influence of a single data point
# in the rbf kernel.
param_grids = {
    'PCA': {
        "svc__C": [0.1, 1, 10, 100],
        "svc__gamma": [0.001, 0.01, 0.1, 1]
    },
    'LDA': {
        "svc__C": [0.1, 1, 10, 100, 200],
        "svc__gamma": [0.001, 0.01, 0.1, 1]
    },
}

Nach der Definition der grundlegenden Abläufe und Werte kann die eigentliche Hyperparameteroptimierung stattfinden. Dafür wird für jede Pipeline eine GridSearch mit 4-fold-Kreuzvalidierung durchgeführt und das beste Ergebnis zählt.

In [ ]:
from sklearn.model_selection import GridSearchCV

best_estimators = {}
best_scores = {}

for method, pipeline in pipelines.items():
    print(f"\nStart training with {method} and hyperparameter optimisation...")
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[method],
        scoring="accuracy",
        cv=4,  # 4-Fold Cross Validation
        verbose=1,
        n_jobs=-1,  # Parallelise the processing of the folds
    )
    grid_search.fit(features, labels)
    best_estimators[method] = grid_search.best_estimator_
    best_scores[method] = grid_search.best_score_
    print(f"\nBest parameters for {method}: {grid_search.best_params_}")
    print(f"Best Cross Validation Avg Accuracy for {method}: {grid_search.best_score_:.4f}")

Das Ergebnis der Analyse zeigt, dass die PCA besser abschneidet. Der optimale Hyperparameter $C = 1$ weist darauf hin, dass die SVM versucht, eine "weiche" Trennlinie zwischen den Klassen zu ziehen - das ist aufgrund der schwierigen Trennbarkeit der Daten ein gutes Zeichen. Das moderat gewählte $\gamma = 0.1$ ermöglicht es der SVM, auf Details einzugehen, ohne zu sehr ins Overfitting zu kommen.

Das auf diese Weise gefundene beste Modell wird nun als optimiertes Modell weiterverwendet und einer finalen Kreuzvalidierung unterzogen (die dasselbe Ergebnis haben wird wie zuvor, sofern immer noch 4 folds verwendet werden), um konkrete Werte für den Vergleich mit dem als nächstes implementierten k-Nearest-Neighbour-Klassifikator zu erhalten.

In [ ]:
from sklearn.model_selection import cross_val_score

optimised_model = best_estimators['PCA']

cv_scores = cross_val_score(optimised_model, features, labels, cv=4, scoring="accuracy")

print("\nResults of the Cross Validation:")
for fold_index, score in enumerate(cv_scores, 1):
    print(f"Fold {fold_index}: Accuracy = {score:.4f}")

print(
    f"\nAverage Accuracy over all folds: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}"
)

## Implementierung des k-nearest-Neighbour-Klassifikators

Zunächst wird der Klassifikator instanziiert und -- wie auch bei der SVM -- die Grid Search für die Hyperparameteroptimierung vorbereitet.
Danach wird der Klassifikator mit den besten Parametern erneut kreuzvalidiert, um finale Ergebnisse für den Vergleich zu erhalten. Insgesamt ist der Prozess damit sehr ähnlich zur Erstellung und Hyperparameteroptimierung der SVM.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(metric="manhattan", weights="distance", n_jobs=-1)
param_grid_knn = {
    "n_neighbors": [1, 3, 5, 7, 9, 11, 13, 15],
}

# Hyperparameter Optimisation
print("\nStart training with k-NN and hyperparameter optimisation...")
grid_search_knn = GridSearchCV(
    estimator=knn,
    param_grid=param_grid_knn,
    scoring="accuracy",
    cv=4,  # 4-Fold Cross Validation
    verbose=1,
    n_jobs=-1,  # Parallelise processing
)
grid_search_knn.fit(features, labels)

# Save the best model, parameters and values
best_knn_model = grid_search_knn.best_estimator_
best_knn_params = grid_search_knn.best_params_
best_knn_score = grid_search_knn.best_score_

# Print results of optimisation
print(f"\nBest parameters for k-NN: {best_knn_params}")
print(f"Best Cross Validation Avg Accuracy for k-NN: {best_knn_score:.4f}")

Wie bereits bei der Hyperparameteroptimierung der SVM hat sich auch beim k-Nearest-Neighbour-Klassifikator ein moderater Wert als optimal herausgestellt: Mit $k = 13$ werden genügend Nachbarn miteinbezogen, um ausreichende Stabilität gegen Ausreißer zu bieten, ohne stark zu Overfitting zu neigen.

In [ ]:
# Validate final model again (no different results expected than before when using 4 folds)
cv_scores_knn = cross_val_score(
    best_knn_model, features, labels, cv=4, scoring="accuracy"
)

print("\nResults of the Cross Validation for k-NN:")
for fold_index, score in enumerate(cv_scores_knn, 1):
    print(f"Fold {fold_index}: Accuracy = {score:.4f}")

print(
    f"\nAverage Accuracy over all folds: {cv_scores_knn.mean():.4f} ± {cv_scores_knn.std():.4f}"
)

## Vergleich der Ergebnisse von SVM und k-NN-Klassifikator

Die SVM schneidet in der 4-fold-Kreuzvalidierung nach der Hyperparameteroptimierung mit einer durchschnittlichen Genauigkeit von 40,5 % etwas besser ab als der k-Nearest-Neighbour-Klassifikator (37,6 %).